<a href="https://colab.research.google.com/github/Nidzoki/PMFSTIntroToDS/blob/main/w11/w11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

### 2025./2026.

# UVOD U PODATKOVNU ZNANOST
## **Domaća zadaća  11**
#### Zadaci za samostalan rad
*analiza sentimenta*

# **Zadatak 1.**
Za analizu sentimenta koristimo [Twitter US Airline Sentiment](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment) skup podataka koji sadrži sentimente vezane za američke zrakoplovne tvrtke.

* Učitajte `Tweets.csv`.
* Iz učitanog skupa podataka ostavite stupce: *airline_sentiment*, *airline*, *name*, *text*, *tweet_created*.
Stvorite novi stupac `date` koristeći stupac *tweet_created*, ali da sadrži samo datum objave tweet-a (bez vremena).

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "Tweets.csv"

df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "crowdflower/twitter-airline-sentiment",
  file_path,
)

print("First 5 records:", df.head())

Using Colab cache for faster access to the 'twitter-airline-sentiment' dataset.
First 5 records:              tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0         

# **Zadatak 2.**
Napravite EDA nad skupom podataka:
* koliko tweetova ima za svaki sentiment, i koji je to postotak od ukupnog broja?
* koliko tweetova ima za svaku airline, i koji je to postotak od ukupnog broja?
* za svaku airline pronađite broj pozitivnih, neutralnih i negativnih tweetova. Koja airline ima najviše pozitivnih, a koja najviše negativnih sentimenata?
* koliko različitih datuma pokriva skup podataka? Koliko ima tweetova za svaki datum i koji je to postotak od ukupnog broja tweetova?
* ...

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [59]:
df['airline_sentiment'].value_counts(), df['airline_sentiment'].value_counts(normalize=True)

(airline_sentiment
 negative    9178
 neutral     3099
 positive    2363
 Name: count, dtype: int64,
 airline_sentiment
 negative    0.626913
 neutral     0.211680
 positive    0.161407
 Name: proportion, dtype: float64)

In [60]:
df['airline'].value_counts(), df['airline'].value_counts(normalize=True)

(airline
 United            3822
 US Airways        2913
 American          2759
 Southwest         2420
 Delta             2222
 Virgin America     504
 Name: count, dtype: int64,
 airline
 United            0.261066
 US Airways        0.198975
 American          0.188456
 Southwest         0.165301
 Delta             0.151776
 Virgin America    0.034426
 Name: proportion, dtype: float64)

In [61]:
df.groupby(['airline', 'airline_sentiment']).size()

airline         airline_sentiment
American        negative             1960
                neutral               463
                positive              336
Delta           negative              955
                neutral               723
                positive              544
Southwest       negative             1186
                neutral               664
                positive              570
US Airways      negative             2263
                neutral               381
                positive              269
United          negative             2633
                neutral               697
                positive              492
Virgin America  negative              181
                neutral               171
                positive              152
dtype: int64

In [62]:
df['date'] = pd.to_datetime(df['tweet_created']).dt.date

In [63]:
df['date'].value_counts(), df['date'].value_counts(normalize=True)

(date
 2015-02-22    3079
 2015-02-23    3028
 2015-02-21    1557
 2015-02-20    1500
 2015-02-17    1408
 2015-02-19    1376
 2015-02-24    1344
 2015-02-18    1344
 2015-02-16       4
 Name: count, dtype: int64,
 date
 2015-02-22    0.210314
 2015-02-23    0.206831
 2015-02-21    0.106352
 2015-02-20    0.102459
 2015-02-17    0.096175
 2015-02-19    0.093989
 2015-02-24    0.091803
 2015-02-18    0.091803
 2015-02-16    0.000273
 Name: proportion, dtype: float64)

# **Zadatak 3.**

Pripremite tekst za automatsku obradu.
* uklonite razmake koji su viška (početak/kraj tweeta)
* prebacite sve tweetove u mala slova
* uklonite korisnička imena (obično započinju s @)

In [64]:
import re

In [65]:
df['text_clean'] = df['text'].str.strip()
df['text_clean'] = df['text_clean'].str.lower()
df['text_clean'] = df['text_clean'].str.replace(r'@\w+', '', regex=True)

In [66]:
df['text_clean'].head()

,text_clean
0,what said.
1,plus you've added commercials to the experien...
2,i didn't today... must mean i need to take an...
3,"it's really aggressive to blast obnoxious ""en..."
4,and it's a really big bad thing about it


# **Zadatak 4.**

Napravite analizu sentimenta nepročišćenog (originalnog) i pročišćenog teksta koristeći alat **VADER**.

Izračunajte 4 klasifikacijske mjere (točnost, preciznost, odziv, f1 mjera) te usporedite rezultate nad originalnim te nad pročišćenim tekstom.

In [67]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [68]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [69]:
df['clean_sentiment_class'] = df['text_clean'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df['original_sentiment_class'] = df['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

In [70]:
def categorize(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [71]:
df['clean_sentiment_class'] = df['clean_sentiment_class'].apply(categorize)
df['original_sentiment_class'] = df['original_sentiment_class'].apply(categorize)

In [72]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [77]:
print(f"Original Acc:{accuracy_score(df['airline_sentiment'], df['original_sentiment_class'])}")
print(f"Clean Acc:{accuracy_score(df['airline_sentiment'], df['clean_sentiment_class'])}")

print(f"Original Precision:{precision_score(df['airline_sentiment'], df['original_sentiment_class'], average='weighted')}")
print(f"Clean Precision:{precision_score(df['airline_sentiment'], df['clean_sentiment_class'], average='weighted')}")

print(f"Original Recall:{recall_score(df['airline_sentiment'], df['original_sentiment_class'], average='weighted')}")
print(f"Clean Recall:{recall_score(df['airline_sentiment'], df['clean_sentiment_class'], average='weighted')}")

print(f"Original F1:{f1_score(df['airline_sentiment'], df['original_sentiment_class'], average='weighted')}")
print(f"Clean F1:{f1_score(df['airline_sentiment'], df['clean_sentiment_class'], average='weighted')}")

Original Acc:0.5428961748633879
Clean Acc:0.5424180327868853
Original Precision:0.6986046019392829
Clean Precision:0.6984147452047103
Original Recall:0.5428961748633879
Clean Recall:0.5424180327868853
Original F1:0.5653990567655425
Clean F1:0.5649699466348022


# **Zadatak 5.**

U ovom zadatku ćete upoznati još jedan leksički pristup analizi sentimenta - [**TextBlob**](https://textblob.readthedocs.io/en/dev/).

Napravite analizu sentimenta nepročišćenog (originalnog) i pročišćenog teksta koristeći alat **TextBlob**.
> [Više o analizi sentimenta u TextBlobu](https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis) - TextBlob objekt ima svojstvo `.sentiment.polarity` kojeg možete poistovijetiti s VADER-ovim `compound` svojstvom.



Izračunajte 4 klasifikacijske mjere (točnost, preciznost, odziv, f1 mjera) te usporedite rezultate nad originalnim te nad pročišćenim tekstom.


In [79]:
from textblob import TextBlob

In [83]:
def get_textblob_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity >= 0.05:
        return 'positive'
    elif polarity <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['pred_original'] = df['text'].apply(get_textblob_sentiment)
df['pred_clean'] = df['text_clean'].apply(get_textblob_sentiment)

In [84]:
print(f"Original Acc:{accuracy_score(df['airline_sentiment'], df['pred_original'])}")
print(f"Clean Acc:{accuracy_score(df['airline_sentiment'], df['pred_clean'])}")

print(f"Original Precision:{precision_score(df['airline_sentiment'], df['pred_original'], average='weighted')}")
print(f"Clean Precision:{precision_score(df['airline_sentiment'], df['pred_clean'], average='weighted')}")

print(f"Original Recall:{recall_score(df['airline_sentiment'], df['pred_original'], average='weighted')}")
print(f"Clean Recall:{recall_score(df['airline_sentiment'], df['pred_clean'], average='weighted')}")

print(f"Original F1:{f1_score(df['airline_sentiment'], df['pred_original'], average='weighted')}")
print(f"Clean F1:{f1_score(df['airline_sentiment'], df['pred_clean'], average='weighted')}")

Original Acc:0.4527322404371585
Clean Acc:0.45259562841530054
Original Precision:0.6752035360685649
Clean Precision:0.6748298519485161
Original Recall:0.45273224043715854
Clean Recall:0.45259562841530054
Original F1:0.46137223622146145
Clean F1:0.4613345324020875


# **Zadatak 6.**

***Razmislite!***

* Ako rezultati oba alata značajno variraju između analize nepročišćenog i pročišćenog teksta, što mislite da bi mogao biti uzrok tih razlika?

* S druge strane, ako rezultati oba alata ostaju gotovo nepromijenjeni bez obzira na to je li tekst pročišćen ili ne, što bi mogao biti razlog takvog ishoda?

### Zašto bi rezultati varirali? <br>
- Gubitak konteksta: Čišćenjem smo možda obrisali bitne znakove uz username (npr. @user :) ili @user !!!).

- Kriva interpretacija: Alat je možda pokušavao čitati riječi unutar imena (npr. @SretnaAna), pa je brisanjem nestao taj lažni "pozitivni" signal.

### Zašto bi ostali isti?
- Leksički ignore: Modeli (VADER, TextBlob) rade na bazi rječnika. Budući da @username nije u rječniku, oni ga već vide kao neutralno (0), pa njegovo brisanje ne mijenja matematiku.

- Robustnost: Alati su već "istrenirani" da ignoriraju šum s društvenih mreža.

### Zaključak:
 Ako su rezultati isti, alat je pametno ignorirao nebitno. Ako variraju, čišćenje je bilo previše agresivno i narušilo je samu poruku.